Now we know that matrix multiplication stetches and rotates the space. Try to image that you take any vector in space and repeatedly multiply it by the matrix. After some iterations this vector will align with rotation axis. Make sure you understood the claim before jumping into implementation.

One caviat is that this vector will get really long and as we know big numbers are computationaly unstable, thus we will simply normalize the vector each time. This trick is called *power itearation* and can be expressed as:

$$b_{k+1} = \frac{Ab_k}{||Ab_k||}.$$

In [140]:
import numpy as np

Let's start with random matrix and compute svd using numpy.

In [292]:
n, m = 4, 3

A = np.random.normal(size=(n, m))
U, S, V = np.linalg.svd(A)

Note, that we can drop some rows/columns without loosing any valuable information.

In [299]:
k = min(n, m)
U = U[:, :k]
V = V[:k]
assert np.allclose(A, U * S @ V)

Finding the first singular value.

In [300]:
v = np.random.normal(size=m)

for _ in range(1000):
    v = A.T @ A @ v / np.linalg.norm(A.T @ A @ v)

s = np.linalg.norm(A @ v)

assert np.allclose(s, S[0])
# Note, that sign of the vector can be flipped
assert np.allclose(V[0], v) or np.allclose(V[0], -v)

*Task:* improve run time by refactoring `A.T @ A` and repeating loop only untill value of `v` becomes stable.

Also we can extract u as follows

In [301]:
u = A @ v / s

assert np.allclose(U[:, 0], u) or np.allclose(U[:, 0], -u)

Now we just need to repeat this.

In [302]:
U_approx = np.zeros((n, k))
S_approx = np.zeros(k)
V_approx = np.zeros((k, m))

# We will modify matrix, thus let's copy it first
B = A.copy()

for i in range(k):

    v = np.random.normal(size=m)
    for _ in range(1000):
        v = B.T @ B @ v / np.linalg.norm(B.T @ B @ v)

    s = np.linalg.norm(B @ v)
    u = B @ v / s     
    
    V_approx[i] = v
    S_approx[i] = s
    U_approx[:, i] = u

    # Trick is to modify original matrix by subtacting dominating component
    B -= s * np.outer(u, v)

Let's check our result

In [303]:
assert np.allclose(S_approx, S)
assert np.allclose(np.abs(V_approx), np.abs(V))
assert np.allclose(np.abs(U_approx), np.abs(U))

Simple!

# (re)sources

Inspired by [blog post](https://towardsdatascience.com/simple-svd-algorithms-13291ad2eef2#:~:text=General%20formula%20of%20SVD%20is,columns%20are%20left%20singular%20vectors) and [wiki](https://en.wikipedia.org/wiki/Power_iteration).